### Basic usage
First we need to import the `Scenario` class which will be the central point for managing a given.. scenario.
Here's how to use it

In [1]:
from powersimdata import Scenario
scenario = Scenario(403)

SCENARIO: test | EasternBase_2020_3

--> State
analyze
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading ct


### Note on documentation
See the [module index](https://breakthrough-energy.github.io/docs/py-modindex.html) on our docs website for a full listing of the code.
There is also interactive help available:

In [2]:
# see the docstring
scenario?

Type:           Scenario
String form:    <powersimdata.scenario.scenario.Scenario object at 0x7f010b2902b0>
File:           /PowerSimData/powersimdata/scenario/scenario.py
Docstring:     
Handles scenario.

:param int/str descriptor: scenario name or index. If None, default to a Scenario
    in Create state.
Init docstring: Constructor.


In [3]:
scenario.print_scenario_info()

--------------------
SCENARIO INFORMATION
--------------------
id: 403
plan: test
name: EasternBase_2020_3
state: analyze
grid_model: usa_tamu
interconnect: Eastern
base_demand: vJan2021
base_hydro: vJan2021
base_solar: vJan2021
base_wind: vJan2021
change_table: Yes
start_date: 2016-01-01 00:00:00
end_date: 2016-12-31 23:00:00
interval: 24H
engine: REISE
runtime: 18:21
infeasibilities: 


### Load base scenario grid data and results

In [4]:
ct= scenario.get_ct()
grid = scenario.get_grid()
load_shed= scenario.get_load_shed()
congu= scenario.get_congu() # get positive direction hourly branch congestion
congl= scenario.get_congl() # get negative direction hourly branch congestion

--> Loading LOAD_SHED
--> Loading CONGU
--> Loading CONGL


### Congestion analysis

In [5]:
congu_yearly = (congu.sum(axis = 0)[congu.sum(axis = 0)!=0]).sort_values(ascending=False) # annual positive direction congestion
congl_yearly = (congl.sum(axis = 0)[congl.sum(axis = 0)!=0]).sort_values(ascending=False) # annual negative direction congestion

In [8]:
# get top 20
congu_yearly.iloc[:20]

30047    394346.155999
43103    324504.754447
8973     313437.994226
43278    280681.694853
24152    252591.721969
34790    237769.241598
80812    184135.303565
20841    177272.959496
42738    158910.326631
80978    131596.446350
12926    116304.061299
4000     115383.018273
28110    107654.025741
30282    104266.056269
77583    103299.358386
28111    102889.090650
56895    101744.622383
30284     99697.458185
30283     98761.233481
25854     96522.012281
dtype: float64

In [9]:
# get top 20
congl_yearly.iloc[:20]

68928    373657.604083
15393    303701.815995
30555    281390.617576
24881    247248.211137
16021    230037.781552
23704    180174.114957
23706    175545.612581
23705    168633.291674
43758    140073.225922
40573    137532.438947
64416    128007.277287
67619    123810.944514
81096    119635.455634
6326     118930.565083
26180    118487.715343
81102    101291.989409
12494    100483.167231
26733    100414.824176
33762     93631.609502
28115     91816.959081
dtype: float64

In [10]:
# get annual congestion on branch 68928,
# which is close to MTEP16 MISO Market Efficiency Project.
congl_yearly.at[68928]

373657.604083

In [11]:
congl[68928] # get hourly congestion on branch 68928

UTC
2016-01-01 00:00:00    83.128237
2016-01-01 01:00:00    82.795075
2016-01-01 02:00:00    81.270021
2016-01-01 03:00:00    80.950065
2016-01-01 04:00:00    80.245885
                         ...    
2016-12-31 19:00:00     0.000000
2016-12-31 20:00:00     0.000000
2016-12-31 21:00:00     0.000000
2016-12-31 22:00:00     0.000000
2016-12-31 23:00:00     0.000000
Freq: <pandas._libs.properties.CachedProperty object at 0x7f00f16a5200>, Name: 68928, Length: 8784, dtype: Sparse[float64, 0]

In [12]:
congl[68928].astype(bool).sum(axis=0) # get annual total binding hours on 68928

4775

In [13]:
grid.branch.loc[68928] # review information of branch 68928

from_bus_id              54070
to_bus_id                54057
r                      0.05092
x                     0.110111
b                      0.00514
rateA                    13.66
rateB                      0.0
rateC                      0.0
ratio                      0.0
angle                      0.0
status                       1
angmin                     0.0
angmax                     0.0
Pf                       -4.07
Qf                         2.3
Pt                        4.08
Qt                       -2.83
mu_Sf                      0.0
mu_St                      0.0
mu_angmin                  0.0
mu_angmax                  0.0
branch_device_type        Line
interconnect           Eastern
from_zone_id                39
to_zone_id                  39
from_zone_name            Iowa
to_zone_name              Iowa
from_lat               42.2985
from_lon              -95.1204
to_lat                 42.3388
to_lon                -95.2962
Name: 68928, dtype: object

In [14]:
grid.bus.loc[54070] # review from bus information of branch 68928

type                  1
Pd                 2.29
Qd                 0.63
Gs                  0.0
Bs                  0.0
zone_id              39
Vm              1.03942
Va              16.6199
baseKV             69.0
loss_zone             1
Vmax                1.1
Vmin                0.9
lam_P             63.95
lam_Q               0.0
mu_Vmax             0.0
mu_Vmin             0.0
interconnect    Eastern
lat             42.2985
lon            -95.1204
Name: 54070, dtype: object

In [15]:
grid.bus.loc[54057] # review to bus information of branch 68928

type                  1
Pd                  0.0
Qd                  0.0
Gs                  0.0
Bs                  0.0
zone_id              39
Vm              1.03869
Va              16.9273
baseKV             69.0
loss_zone             1
Vmax                1.1
Vmin                0.9
lam_P             63.95
lam_Q               0.0
mu_Vmax             0.0
mu_Vmin             0.0
interconnect    Eastern
lat             42.3388
lon            -95.2962
Name: 54057, dtype: object

### Retrive other types of output

In [ ]:
pg = scenario.get_pg() # get hourly generation output on units
pf_ac = scenario.get_pf() # get hourly power flow for AC lines
pf_dc = scenario.get_dcline_pf() # get hourly power flow for DC lines
lmp = scenario.get_lmp() # get hourly bus locational marginal price

# not available for this scenario
# e_storage = scenario.get_storage_e() # get energy state of charge of storage (if present in scenario)
# pg_storage = scenario.get_storage_pg() # get generation profile for storage (if present in scenario)

--> Loading PG
--> Loading PF
